In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nltk

In [ ]:
df = pd.read_csv("/kaggle/input/nlp-getting-started/train.csv")

In [ ]:
df.info()

In [ ]:
df.head()

# **EDA: check if text length is a factor for deciding.** #

In [ ]:
df['msg_len'] = df['text'].apply(len)
df['msg_len']

In [ ]:
df['msg_len'].describe()

In [ ]:
df[df['msg_len']==157]['text'].iloc[0]

In [ ]:
sns.barplot(x=df['target'], y=df['msg_len'],data=df)

In [ ]:
sns.set_style('darkgrid')
df.hist(column='msg_len', by = 'target', sharey=True, bins = 50, figsize=(12,4))

In [ ]:
df.drop(['id','keyword','location','msg_len'],axis=1,inplace=True)

In [ ]:
import string
import time
from nltk.corpus import stopwords
commonwords = stopwords.words('english') 
from nltk.corpus import words
import re
from nltk.stem.porter import PorterStemmer
string.punctuation

In [ ]:
corpus =[]
for i in range(0,df.shape[0]):
    review = re.sub('[^a-zA-Z]'," ",df['text'][i])
    review = review.lower()
    review = review.split()
    ps = PorterStemmer()
    review = " ".join(review)
    corpus.append(review)
time.sleep(3)
    

In [ ]:
corpus[0:5]

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
X = cv.fit_transform(corpus).toarray()
y = df.iloc[:,-1].values

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC


nb_model = MultinomialNB()
nb_model.fit(X_train,y_train)
nb_preds = nb_model.predict(X_test)

from sklearn.metrics import classification_report,accuracy_score

print(classification_report(y_test,nb_preds))
print("accuracy is:", round(accuracy_score(y_test,nb_preds)*100),"%")



In [ ]:
lr_model = LogisticRegression(random_state=101, max_iter=1000)
lr_model.fit(X_train,y_train)
lr_preds = lr_model.predict(X_test)

print("lr",classification_report(y_test,lr_preds))
print("lr_accuracy is:", round(accuracy_score(y_test,lr_preds)*100),"%")



In [ ]:
svc_model = LinearSVC()
svc_model.fit(X_train,y_train)
svc_preds = svc_model.predict(X_test)


print("svc",classification_report(y_test,svc_preds))
print("svc_accuracy is:", round(accuracy_score(y_test,svc_preds)*100),"%")

In [ ]:
dft = pd.read_csv("/kaggle/input/nlp-getting-started/test.csv")
dft.drop(['keyword','location','id'],axis=1,inplace=True)
dft.head(3)

In [ ]:
test_corpus =[]
for i in range(0,dft.shape[0]):
    r = re.sub('[^a-zA-Z]'," ", dft['text'][i])
    r = r.lower()
    r = r.split()
    ps = PorterStemmer()
    r = " ".join(r)
    test_corpus.append(r)
time.sleep(3)
    

In [ ]:
test_corpus[0:5]

In [ ]:
test_data = cv.transform(test_corpus)

In [ ]:
op = pd.read_csv("/kaggle/input/nlp-getting-started/sample_submission.csv")
op['target']= lr_model.predict(test_data)
op['target'].value_counts()

In [ ]:
op.head()

In [ ]:
op.to_csv('Submissions.csv',index=False)